In [1]:
from tensorflow_docs.vis import embed
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,Dropout,GlobalMaxPool1D
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical,plot_model
import time
import pandas as pd
import numpy as np
import imageio
from numpy import random
import cv2 as cv
import pafy
import os
%matplotlib inline

import datetime as dt 
from collections import deque
from moviepy.editor import *
import pafy
import random
import time

from create_dataset import *



2022-03-16 16:59:17.788845: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-16 16:59:17.819943: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-16 16:59:17.820309: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-16 16:59:17.821517: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [2]:
import build_model

In [3]:
vit_model=build_model.get_compiled_model()

In [4]:
vit_model.load_weights('saved_model')

2022-03-16 16:59:50.600526: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open saved_model: FAILED_PRECONDITION: saved_model; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


In [5]:
feature_extractor=features_extraction_model()


In [6]:
X_test=build_model.X_test
y_test=build_model.y_test
class_names=CLASSES_LIST

In [7]:
IMG_SIZE=resized_height
seq_len=seq_len

In [8]:
vit_model.evaluate(X_test,y_test,verbose=1)

2022-03-16 17:00:08.481045: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


2/2 [==============================] - 2s 9ms/step - loss: 0.2474 - accuracy: 0.9762


2022-03-16 17:00:08.689143: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8300


[0.24735523760318756, 0.976190447807312]

In [9]:
test_video_dire='test_video.mp4'

In [16]:
def predict_on_untrimmed(video_dire,output_file_path,seq_len):
    video_reader=cv.VideoCapture(video_dire)
    vides_total_frame=int(video_reader.get(cv.CAP_PROP_FRAME_COUNT))
    origianal_width=int(video_reader.get(cv.CAP_PROP_FRAME_WIDTH))
    original_height=int(video_reader.get(cv.CAP_PROP_FRAME_HEIGHT))
    video_writer = cv.VideoWriter(output_file_path, cv.VideoWriter_fourcc('M', 'P', '4', 'V'), 
                                   video_reader.get(cv.CAP_PROP_FPS), (origianal_width, original_height))
    frame_queue=deque(maxlen=seq_len)
    predict_class_name=''
    start_time=time.time()
    while video_reader.isOpened():
        success,frame=video_reader.read()
        if not success:
            break
        resize_frame=cv.resize(frame,(IMG_SIZE,IMG_SIZE))    
        frame_queue.append(resize_frame)
        if len(frame_queue)==seq_len:
            frames=np.asarray(frame_queue)
            
            features=feature_extractor.predict(frames).reshape(-1,20,1024)
            probs=vit_model.predict(features)[0]
            text_height=15
            for class_name,prob in zip(class_names,probs):
                text='{}: {:.5f}'.format(class_name,prob)
                cv.putText(frame, text, (10, text_height), cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
                text_height=text_height+35
            
                
            
        video_writer.write(frame)    
        
    end_time=time.time()
    
    print('start_time: {}'.format(start_time))
    print('end_time: {}'.format(end_time))
    print("num_frame: {}".format(vides_total_frame))
    seconds=end_time-start_time
    fps=vides_total_frame/seconds
    print('fps: ',fps)    
    video_reader.release()
    video_writer.release()
     

In [17]:
predict_on_untrimmed(test_video_dire,'demo.mp4',seq_len)

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


start_time: 1647418145.1062174
end_time: 1647418307.6346984
num_frame: 866
fps:  5.328296890713661


In [16]:
frames=np.asarray(frames)

In [17]:
frames.shape

(1, 20, 128, 128, 3)

In [26]:
frame.shape

(240, 320, 3)

In [22]:
test=feature_extractor.predict(frames).reshape(-1,20,1024)

In [25]:
probs=vit_model.predict(test)[0]

In [28]:
for class_name,prob in zip(class_names,probs):
    print("{}: {:.5f}".format(class_name,prob))

WalkingWithDog: 1.00000
TaiChi: 0.00000
Swing: 0.00000
HorseRace: 0.00000


In [ ]:
cv.imshow('',frame)
cv.waitKey(0)
cv.destroyAllWindows()

In [2]:
frame=cv.imread('test.png')

In [3]:
frame.shape

(576, 1024, 3)

In [ ]:
cv.imshow('',frame)
cv.waitKey(0)
cv.destroyAllWindows()